In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

import Odin

from fieldAccess import *
import numpy as np
import MeshConfig

nx = 4
ny = 1

inletVelocity = 0.2

geom = Odin.createGeometry( 'rectangle', [nx,ny] )
mesh = Odin.createMesh( geom, res=1 )

# make just a list, give a name as optional parameter to flow model
# make this a flowmodeles constructor with kwargs here and then remove simulation instance
myFlowModels = {
    'u' : Odin.TransportModels.staggeredTransport_u,
    'v' : Odin.TransportModels.staggeredTransport_v,
    'p' : Odin.PressureModels.Pressure
}

passiveFields = {}

Odin.initialize(flowmodels=myFlowModels, mesh=mesh, geometry=geom, passiveFields=passiveFields )

u=Odin.getField('u')
v=Odin.getField('v')
p=Odin.getField('p')

# relates to viscosity:
u.govModel.setDiffusionCoefficient(1e-2)
v.govModel.setDiffusionCoefficient(1e-2)

u.data.fill(inletVelocity)
Odin.defineBoundaryCondition(field=u, boundaryName='top', type='fixedValue', value=0)
Odin.defineBoundaryCondition(field=u, boundaryName='bottom', type='fixedValue', value=0)
Odin.defineBoundaryCondition(field=u, boundaryName='right', type='zeroGradient')
Odin.defineBoundaryCondition(field=u, boundaryName='left', type='fixedValue', value=inletVelocity)

v.data.fill(0.0)
Odin.defineBoundaryCondition(field=v, boundaryName='top', type='fixedValue', value=0)
Odin.defineBoundaryCondition(field=v, boundaryName='bottom', type='fixedValue', value=0)
Odin.defineBoundaryCondition(field=v, boundaryName='left', type='fixedValue', value=0 )
Odin.defineBoundaryCondition(field=v, boundaryName='right', type='fixedValue', value=0)

p.data.fill(0)
Odin.defineBoundaryCondition(field=p, boundaryName='top', type='freeFlow')
Odin.defineBoundaryCondition(field=p, boundaryName='bottom', type='freeFlow')
Odin.defineBoundaryCondition(field=p, boundaryName='left', type='freeFlow' )
Odin.defineBoundaryCondition(field=p, boundaryName='right', type='constantPressure', value=0)

#sim.display(u,mesh)

# underRelaxation
alphaP = 0.3
alphaU = 0.7
alphaV = alphaU


# make a max function when calculating the d coefficients. some a_p values might be extremely low, such that the corresp d values become hughe

# making a more sophisticated initial guess:
# u.data[internal] = Odin.solve(u)[internal]
# v.data[internal] = Odin.solve(v)[internal]


for i in range(20):

    u.data = Odin.solve(u)
    v.data = Odin.solve(v)

    pc = Odin.solve(p)
    p.data += pc

    u.data[internal_u] += p.govModel.d_u[internal_u] * ( pc[west] - pc[east] )
    v.data[internal_v] += p.govModel.d_v[internal_v] * ( pc[south] - pc[north])



# preparing the simple loop:
for i in range(0):

    uOld = u.data
    vOld = v.data

    uNew = Odin.solve(u)
    vNew = Odin.solve(v)
    pc = Odin.solve(p)

    # correcting pressure
    p.data += alphaP*pc

    totInFlow = Odin.calcFlowRate(field=u, boundaryName='left')
    totOutFlow = Odin.calcFlowRate(field=u, boundaryName='right')

    # correcting internal:    what about the left side?
    uNew[internal_u] += p.govModel.d_u[internal_u] * ( pc[west] - pc[east] )
    vNew[internal_v] += p.govModel.d_v[internal_v] * ( pc[south] - pc[north] )
    uNew[boundary_east] = uOld[boundary_nb1_east] * totInFlow/totOutFlow

    u.data = alphaU*uNew + (1-alphaU)*uOld
    #v.data = alphaV*vNew + (1-alphaV)*vOld

    # correcting boundaries:  Can I not set them algorithmically? this is not nice
    # correcting boundaries a second time:
    totInFlow = Odin.calcFlowRate(field=u, boundaryName='left')
    totOutFlow = Odin.calcFlowRate(field=u, boundaryName='right')

    u.data[boundary_east] = u.data[boundary_nb1_east] * totInFlow/totOutFlow
#    v.data[boundary_east] = v.data[boundary_nb1_east] * totInFlow/totOutFlow

    # totInFlow = Odin.calcFlowRate(field=u, boundaryName='left')
    # totOutFlow = Odin.calcFlowRate(field=u, boundaryName='right')
    #
    #print((totInFlow-totOutFlow)/totInFlow)


totInFlow = Odin.calcFlowRate(field=u, boundaryName='left')
totOutFlow = Odin.calcFlowRate(field=u, boundaryName='right')

print("inflow:\t", totInFlow )
print("outflow:\t", totOutFlow )

Odin.display(u,mesh)
Odin.display(v,mesh)
Odin.display(pc,mesh)
Odin.display(p,mesh)
#sim.display(v,mesh)
#sim.display(p,mesh)
#
#
# print(p.govModel.linSystem.A)
# print(p.govModel.linSystem.b)

ValueError: operands could not be broadcast together with shapes (1,4) (1,5) (1,4) 

In [ ]:

# post-processing:
import numpy as np
import matplotlib.pyplot as plt

nbcellsX = mesh.cells_x
cellSpacing = mesh.uniformSpacing
LenX = geom.lenX

xSim = np.linspace(0+0.5*cellSpacing,LenX-0.5*cellSpacing,nbcellsX+1)
ySim = u.data[ny//2 ,:]

# xTheo = np.linspace(0,LenX, 100)
# S = np.ones(len(xTheo))*heatSource
# yTheo = tempDistr(xTheo)

ax = plt.gca()
ax.plot(xSim, ySim, 'x', label='simulation')
#ax.plot(xTheo, yTheo, label='theoretical')

plt.legend()
plt.show()

print(ySim)


In [ ]:
# shutting the interpreter down, so I can a fresh instance next time.
# my modules are automatically checked for updates
import os
os._exit(00)